<a href="https://colab.research.google.com/github/mneedham/data-science-training/blob/master/03_Recommendations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recommendations

In this notebook we're going to learn how to make recommendations using Neo4j. As with the other notebooks, let's get our environment setup.

In [1]:
!pip install py2neo pandas matplotlib

And let's import those libraries:

In [2]:
from py2neo import Graph
import pandas as pd

import matplotlib 
import matplotlib.pyplot as plt

plt.style.use('fivethirtyeight')
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.max_colwidth', 100)


Update the cell below with the same Sandbox credentials that you used in the first notebook:

In [3]:
# Change the line of code below to use the IP Address, Bolt Port, and Password of your Sandbox.
# graph = Graph("bolt://<IP Address>:<Bolt Port>", auth=("neo4j", "<Password>")) 

# graph = Graph("bolt://18.234.168.45:33679", auth=("neo4j", "daybreak-cosal-rumbles")) 
graph = Graph("bolt://data-science-training-neo4j", auth=("neo4j", "admin")) 

##  Finding popular authors

Since we're going to make collaborator suggestions so let's find authors who have written the most articles so that we have some data to work with.

In [4]:
popular_authors_query = """
MATCH (author:Author)
RETURN author.name, size((author)<-[:AUTHOR]-()) AS articlesPublished
ORDER BY articlesPublished DESC
LIMIT 10
"""

graph.run(popular_authors_query).to_data_frame()

,articlesPublished,author.name
0,89,Peter G. Neumann
1,80,Peter J. Denning
2,72,Moshe Y. Vardi
3,71,Pamela Samuelson
4,65,Bart Preneel
5,56,Vinton G. Cerf
6,53,Barry W. Boehm
7,49,Mark Guzdial
8,47,Edwin R. Hancock
9,46,Josef Kittler


Let's pick one of these authors...

In [5]:
author_name = "Tao Xie"

And let's have a look what articles they've published and how many citations they've received:

In [6]:
author_articles_query = """
MATCH (:Author {name: $authorName})<-[:AUTHOR]-(article)
RETURN article.title AS article, article.year AS year, size((article)<-[:CITED]-()) AS citations
ORDER BY citations DESC
LIMIT 20
"""

graph.run(author_articles_query,  {"authorName": author_name}).to_data_frame()

,article,citations,year
0,An approach to detecting duplicate bug reports using natural language and execution information,20,2008
1,Mining exception-handling rules as sequence association rules,14,2009
2,Inferring method specifications from natural language API descriptions,9,2012
3,Mining API mapping for language migration,9,2010
4,Performance debugging in the large via mining millions of stack traces,8,2012
5,Where do developers log? an empirical study on logging practices in industry,7,2014
6,Precise identification of problems for structural test generation,7,2011
7,Is operator-based mutant selection superior to random mutant selection?,4,2010
8,Teaching and learning programming and software engineering via interactive gaming,4,2013
9,Automatic extraction of abstract-object-state machines from unit-test executions,4,2006


Find the authors collaborators...

In [7]:
collaborations_query = """
MATCH (:Author {name: $authorName})<-[:AUTHOR]-(article)-[:AUTHOR]->(coauthor)
RETURN coauthor.name AS coauthor, count(*) AS collaborations
ORDER BY collaborations DESC
LIMIT 10
"""

graph.run(collaborations_query,  {"authorName": author_name}).to_data_frame()

,coauthor,collaborations
0,Nikolai Tillmann,11
1,Jonathan de Halleux,7
2,Lu Zhang,6
3,Xusheng Xiao,5
4,Dongmei Zhang,5
5,Hong Mei,4
6,Xiaoyin Wang,4
7,Evan Martin,4
8,Judith Bishop,3
9,Wolfram Schulte,3


How would we suggest some future collaborators for this author? One way is by looking at the collaborators of their collaborators!

In [8]:
collaborations_query = """
MATCH (author:Author {name: $authorName})<-[:AUTHOR]-(article)-[:AUTHOR]->(coauthor),
      (coauthor)<-[:AUTHOR]-()-[:AUTHOR]->(coc)
WHERE not((coc)<-[:AUTHOR]-()-[:AUTHOR]->(author)) AND coc <> author      
RETURN coc.name AS coauthor, count(*) AS collaborations
ORDER BY collaborations DESC
LIMIT 10
"""

graph.run(collaborations_query,  {"authorName": author_name}).to_data_frame()

,coauthor,collaborations
0,Mike Barnett,37
1,Yingfei Xiong,36
2,Hongyu Zhang,32
3,Lingming Zhang,32
4,Bing Xie,30
5,Christoph Csallner,22
6,Patrice Godefroid,22
7,Gregg Rothermel,22
8,Willem Visser,22
9,Weizhong Shao,19


What about if an author wanted to find articles, with articles close to their own work showing up first.

In [14]:
query = """
CALL db.index.fulltext.createNodeIndex('articles', ['Article'], ['title', 'abstract'])
graph.run(query)
"""

In [15]:
query = """
MATCH (a:Article)-[:AUTHOR]->(author:Author)
WHERE author.name=$authorName
WITH author, collect(a) as articles
CALL algo.pageRank.stream(
  'CALL db.index.fulltext.queryNodes("articles", $searchTerm)
   YIELD node, score
   RETURN id(node) as id',
  'MATCH (a1:Article)-[:CITED]->(a2:Article) 
   RETURN id(a1) as source,id(a2) as target', 
  {sourceNodes: articles,graph:'cypher', params: {searchTerm: $searchTerm}})
YIELD nodeId, score
WITH algo.getNodeById(nodeId) AS n, score
WHERE not(exists((author)<-[:AUTHOR]-(n)))
RETURN n.title as article, score, [(n)-[:AUTHOR]->(author) | author.name][..5] AS authors
order by score desc limit 10
"""

params = {"authorName": "Tao Xie", "searchTerm": "open source"}
graph.run(query, params).to_data_frame()

,article,authors,score
0,Static detection of cross-site scripting vulnerabilities,"[Zhendong Su, Gary Wassermann]",0.236
1,Characterizing logging practices in open-source software,"[Ding Yuan, Soyeon Park, Yuanyuan Zhou]",0.128
2,"Automated, contract-based user testing of commercial-off-the-shelf components","[Lionel C. Briand, Yvan Labiche, Michal M. Sówka]",0.128
3,Concern graphs: finding and describing concerns using structural program dependencies,"[Gail C. Murphy, Martin P. Robillard]",0.128
4,Who should fix this bug,"[Lyndon Hiew, John Anvik, Gail C. Murphy]",0.128
5,Conceptual module querying for software reengineering,"[Gail C. Murphy, Elisa L. A. Baniassad]",0.108
6,IBM's pragmatic embrace of open source,[Pamela Samuelson],0.000
7,Open courseware and open source software,"[Stefan Baldi, Anett Mehler-Bicher, Hauke Heier]",0.000
8,Reusing Open-Source Software and Practices: The Impact of Open-Source on Commercial Vendors,"[Alan W. Brown, Grady Booch]",0.000
9,From Research Software to Open Source,[Susan L. Graham],0.000


In [16]:
params = {"authorName": "Margus Veanes", "searchTerm": "open source"}
graph.run(query, params).to_data_frame()

,article,authors,score
0,"TEG: A High-Performance, Scalable, Multi-network Point-to-Point Communications Methodology","[Ralph H. Castain, Mitchel W. Sukalski, Edgar Gabriel, Graham E. Fagg, Jack Dongarra]",3.319
1,"Open MPI: Goals, Concept, and Design of a Next Generation MPI Implementation","[Edgar Gabriel, Vishal Sahay, Timothy S. Woodall, Jack Dongarra, Jeffrey M. Squyres]",2.870
2,Concern graphs: finding and describing concerns using structural program dependencies,"[Gail C. Murphy, Martin P. Robillard]",2.626
3,Conceptual module querying for software reengineering,"[Gail C. Murphy, Elisa L. A. Baniassad]",2.542
4,Who should fix this bug,"[Lyndon Hiew, John Anvik, Gail C. Murphy]",2.342
5,Hipikat: recommending pertinent software development artifacts,"[Gail C. Murphy, Davor Cubranic]",2.205
6,Version Sensitive Editing: Change History as a Programming Tool,[David L. Atkins],2.151
7,DECKARD: Scalable and Accurate Tree-Based Detection of Code Clones,"[Stéphane Glondu, Zhendong Su, Lingxiao Jiang, Ghassan Misherghi]",2.145
8,Recovering documentation-to-source-code traceability links using latent semantic indexing,"[Andrian Marcus, Jonathan I. Maletic]",2.006
9,Coverage is not strongly correlated with test suite effectiveness,"[Laura Inozemtseva, Reid Holmes]",1.928
